In [70]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
print('Libraries Imported')

Libraries Imported


In [71]:
df_iems=pd.read_csv('crinacle_iem_rankings.csv')
df_iems.sample(2)

,Rank,Value Rating,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status,Ranksort,Tonesort,Techsort,Pricesort
2040,D,NaN,Kojo KJB-01 “Keyagu”,910,V-shaped,NaN,D,C-,DD,e-Earphone demo unit,316,183,133,910
141,B,NaN,Unique Melody Martian,700,Neutral with bass boost,Well-tuned hybrid with excellent bass response...,A-,B,2BA 2DD,Stereo SG demo unit,88,31,57,700


EDA

In [72]:
print(df_iems.shape)
df_iems.info()

(2143, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             2143 non-null   object
 1   Value Rating     155 non-null    object
 2   Model            2143 non-null   object
 3   Price (MSRP)     2141 non-null   object
 4   Signature        2141 non-null   object
 5   Comments         1793 non-null   object
 6   Tone Grade       2141 non-null   object
 7   Technical Grade  2141 non-null   object
 8   Setup            2135 non-null   object
 9   Status           2125 non-null   object
 10  Ranksort         2143 non-null   object
 11  Tonesort         2143 non-null   object
 12  Techsort         2143 non-null   object
 13  Pricesort        2143 non-null   object
dtypes: object(14)
memory usage: 234.5+ KB


In [73]:
print(df_iems['Value Rating'].count())
df_iems.isnull().sum()

155


Rank                  0
Value Rating       1988
Model                 0
Price (MSRP)          2
Signature             2
Comments            350
Tone Grade            2
Technical Grade       2
Setup                 8
Status               18
Ranksort              0
Tonesort              0
Techsort              0
Pricesort             0
dtype: int64

Drop the "Value Rating" as most of it is null

In [74]:
df_iems=df_iems.drop(columns='Value Rating')
df_iems.sample(2)

,Rank,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status,Ranksort,Tonesort,Techsort,Pricesort
2127,E,Sensaphonics 3MAX,1050,Warm,It's a slight improvement over the 2MAX but no...,E,D,3BA,Jaben Melbourne demo unit,605,422,183,1050
1299,B-,Sony WF-1000XM3,230,Neutral with bass boost,"Proper tonality, solid technicalities yet with...",B+,B-,DD,ConnectIT demo unit,116,43,73,230


Drop all unecessary hidden column from the original data

In [75]:
df_iems=df_iems.drop(columns=['Ranksort','Tonesort','Techsort','Pricesort'])
df_iems.sample(2)

,Rank,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status
432,C+,Ultimate Ears UE7 Pro,850,Warm neutral,"Good clarity and detail with boosted, warm bas...",B-,C+,3BA,Zeppelin & Co demo unit
527,C,FiiO F3,25,U-shaped,Well-balanced U-shaped sound that's rather aty...,B-,C,DD,E1 demo unit


Exploring Price Column

In [76]:
#Renaming it for better calling

df_iems.rename({'Price (MSRP)':'Price'}, axis=1, inplace=True)
df_iems.sample(2)

,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status
1726,C,QKZ VK4,20,V-shaped,Stealth-revised. Latest version is just anothe...,C,C,DD,Third party unit
1638,C,Shuoer D13,120,V-shaped,NaN,C,C+,DD,Available for demo at The Hangout


In [77]:
#Convert the price column to numeric, non number are marked as null
df_iems['Price']=pd.to_numeric(df_iems['Price'], errors='coerce') #stored it in float
df_iems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rank             2143 non-null   object 
 1   Model            2143 non-null   object 
 2   Price            1886 non-null   float64
 3   Signature        2141 non-null   object 
 4   Comments         1793 non-null   object 
 5   Tone Grade       2141 non-null   object 
 6   Technical Grade  2141 non-null   object 
 7   Setup            2135 non-null   object 
 8   Status           2125 non-null   object 
dtypes: float64(1), object(8)
memory usage: 150.8+ KB


In [88]:
df_iems['Price'].isnull().sum()
df_iems.dropna(subset=['Price'], inplace=True)
display(df_iems.shape)
df_iems.isna().sum()


(1886, 9)

Rank                 0
Model                0
Price                0
Signature            0
Comments           336
Tone Grade           0
Technical Grade      0
Setup                4
Status              12
dtype: int64

In [92]:
warnings.filterwarnings('ignore')
df_iems['Comments'].fillna('No Comments', inplace=True)
df_iems['Status'].fillna('No Status', inplace=True)
df_iems.isna().sum()

Rank               0
Model              0
Price              0
Signature          0
Comments           0
Tone Grade         0
Technical Grade    0
Setup              4
Status             0
dtype: int64

Treating Setup column